## Imports

In [0]:
!pip install spotipy
# https://spotipy.readthedocs.io/en/2.12.0/

In [0]:
!pip install spotify

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 1.2MB 4.8MB/s 
     |████████████████████████████████| 153kB 8.0MB/s 
     |████████████████████████████████| 256kB 8.8MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=5bc58bcdf895a2e82fed4358c87b2ae0f6351ef5a28f60664860f67be5187013
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [0]:
import pandas as pd
import numpy as np

import spotipy
from spotipy import oauth2, Spotify

from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, LabelEncoder

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.preprocessing import scale
from tensorflow.keras.optimizers import Adam

import joblib

## Spotify Authentication

In [0]:
# Spotify API authentication
cid ='555816fae63a45f29f50f99bbc81393d'
secret ='008dc4e8c3e3470c97392be068578540'

credentials = oauth2.SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

## Track Info Functions

In [0]:
def fetch_info(artist_name, track_name):
  """
  function to retrieve relevant info on user's input from the Spotify API
  user inputs artist name and track name
  fxn gets info including the unique track id, album name, id, and cover image,
  a sample of the track if exists, and audio feature data for the track
  """
  data = sp.search(q=f'artist:{artist_name} track:{track_name}')
  artist_name = data['tracks']['items'][0]['artists'][0]['name']
  track_name = data['tracks']['items'][0]['name']
  track_id = data['tracks']['items'][0]['id']   
  album_name = data['tracks']['items'][0]['album']['name']
  album_id = data['tracks']['items'][0]['album']['id']
  album_cover_link = data['tracks']['items'][0]['album']['images'][0]['url']
  track_sample = data['tracks']['items'][0]['preview_url']
  audio_features = sp.audio_features(track_id)
  audio_features = audio_features[0] # changes the provided list from spotify into a dictionary
  irrelevant = ["id", "uri", "analysis_url", "type", "track_href"]
  for key in irrelevant:
    del audio_features[key]
  #audio_features = jsonify(audio_features) in FLASK app
  return track_name, artist_name, track_id, album_name, album_id, album_cover_link, track_sample, audio_features

In [0]:
fetch_info("Tame Impala", "Borderline")

('Borderline',
 'Tame Impala',
 '5hM5arv9KDbCHS0k9uqwjr',
 'The Slow Rush',
 '31qVWUdRrlb8thMvts0yYL',
 'https://i.scdn.co/image/ab67616d0000b27358267bd34420a00d5cf83a49',
 None,
 {'acousticness': 0.0406,
  'danceability': 0.621,
  'duration_ms': 237800,
  'energy': 0.873,
  'instrumentalness': 9.1e-06,
  'key': 5,
  'liveness': 0.0824,
  'loudness': -3.067,
  'mode': 0,
  'speechiness': 0.0369,
  'tempo': 97.96,
  'time_signature': 4,
  'valence': 0.873})

In [0]:
def track_feat(track_id):
  """
  function to retrieve relevant info from track ID and store it in a df
  """
  track_features = sp.audio_features(track_id)
  track_features = pd.DataFrame(track_features[0], index=[0])
  track_features = track_features.drop(columns=["id", "uri", "analysis_url", "type", "track_href", "duration_ms", "time_signature"])
  return track_features

In [0]:
track_feat('5hM5arv9KDbCHS0k9uqwjr')

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.621,0.873,5,-3.067,0,0.0369,0.0406,0.000009,0.0824,0.873,97.96


## Database


In [0]:
# 5/27 created new dataset with top tracks from the past two years

tracks = pd.read_csv("https://raw.githubusercontent.com/Build-Week-Spotify-Song-Suggester-2/datascience/master/SpotifyTrackFeatures.csv", error_bad_lines=False)
print(tracks.shape)
tracks.head()

(120104, 17)


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373.0,0.339,0.000,1,0.0812,-7.678,1.0,0.4090,203.927,4.0,0.118,44
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800.0,0.557,0.000,8,0.2860,-7.259,1.0,0.4570,159.009,4.0,0.371,10
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913.0,0.723,0.000,9,0.0824,-5.890,0.0,0.0454,114.966,4.0,0.382,63
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381.0,0.579,0.912,5,0.0994,-12.118,0.0,0.0701,123.003,4.0,0.641,9
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016.0,0.792,0.878,7,0.0332,-10.277,1.0,0.0661,120.047,4.0,0.928,8


In [0]:
tracks = pd.DataFrame(tracks)
tracks.dtypes

artist_name          object
track_id             object
track_name           object
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                float64
speechiness         float64
tempo               float64
time_signature      float64
valence             float64
popularity            int64
dtype: object

In [0]:
tracks = tracks.drop_duplicates(['track_id'])
tracks.shape

(119923, 17)

# Neural Network


Neural networks do not work better than traditional statistical algorithms on tabular numeric data, especially of such a small size (~120k rows). But I'm attempting one anyway. 

In [0]:
tracks.head()

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373.0,0.339,0.000,1.0,0.0812,-7.678,1.0,0.4090,203.927,4.0,0.118,44
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800.0,0.557,0.000,8.0,0.2860,-7.259,1.0,0.4570,159.009,4.0,0.371,10
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913.0,0.723,0.000,9.0,0.0824,-5.890,0.0,0.0454,114.966,4.0,0.382,63
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381.0,0.579,0.912,5.0,0.0994,-12.118,0.0,0.0701,123.003,4.0,0.641,9
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016.0,0.792,0.878,7.0,0.0332,-10.277,1.0,0.0661,120.047,4.0,0.928,8


In [0]:
# numeric data
# scale it
numeric_features = ['acousticness','danceability','energy','instrumentalness',
                      'key', 'liveness', 'loudness','mode','speechiness', 'tempo', 'valence']
tracks[numeric_features] = tracks[numeric_features].astype(float)
numeric_features_scaled = scale(tracks[numeric_features])
tracks[numeric_features] = numeric_features_scaled

In [0]:
# encode artists so they can be included 
encoder = LabelEncoder()
enc_feat = encoder.fit_transform(tracks['artist_name'])
tracks['artist_enc'] = enc_feat

In [0]:
# add encoded artists to features to input
data_features = ['artist_enc', 'acousticness','danceability','energy','instrumentalness',
                      'key', 'liveness', 'loudness','mode','speechiness', 'tempo', 'valence']
tracks.dtypes

artist_name          object
track_id             object
track_name           object
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                float64
speechiness         float64
tempo               float64
time_signature      float64
valence             float64
popularity            int64
artists_enc           int64
artist_enc            int64
dtype: object

In [0]:
tracks[data_features] = tracks[data_features].astype(float)
num_features = tracks[data_features].astype(float)
# make all dtypes float
num_features.dtypes

artist_enc          float64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                float64
speechiness         float64
tempo               float64
valence             float64
dtype: object

In [0]:
num_features.shape

(119923, 12)

In [0]:
tracks.shape

(119923, 19)

In [0]:
model = Sequential()
model.add(Dense(input_shape=(num_features.shape[1],), activation='relu',
          units=tracks.shape[0], use_bias=False))
# instantiated simple model

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 119923)            1439076   
Total params: 1,439,076
Trainable params: 1,439,076
Non-trainable params: 0
_________________________________________________________________


In [0]:
# create and normalize vectors
def normie(vect):
  norm_vect = np.linalg.norm(vect, axis=1, keepdims=True)
  return (vect / norm_vect)

In [0]:
norm_track_feat = normie(num_features)
norm_track_feat.head()

,artist_enc,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,1.0,-0.000007,0.000027,-1.144022e-05,5.330098e-07,-0.000036,-0.000021,0.000009,0.000027,0.000076,0.000078,-0.000006
1,1.0,-0.000007,0.000044,-4.822872e-06,5.330098e-07,0.000026,0.000018,0.000011,0.000027,0.000088,0.000038,-0.000006
2,1.0,-0.000009,0.000004,2.901862e-07,7.159827e-07,0.000046,-0.000029,0.000024,-0.000052,-0.000024,-0.000001,-0.000008
3,1.0,-0.000038,0.000214,-2.430428e-05,9.620558e-05,-0.000005,-0.000105,-0.000071,-0.000227,-0.000066,0.000037,-0.000033
4,1.0,-0.000039,0.000197,1.351489e-05,9.273520e-05,0.000099,-0.000180,-0.000018,0.000157,-0.000072,0.000022,-0.000033


In [0]:
# get weights
# use matrix of the normalized track features
model.set_weights([np.array(norm_track_feat.T)])
model.get_weights()

[array([[ 1.0000000e+00,  1.0000000e+00,  1.0000000e+00, ...,
          9.9999958e-01,  9.9991834e-01,  9.9999988e-01],
        [-6.5670524e-06, -6.5167451e-06, -8.7516455e-06, ...,
          5.3183781e-04,  4.8972564e-03,  1.8833340e-04],
        [ 2.6860635e-05,  4.4299515e-05,  4.2411484e-06, ...,
          7.4756426e-05, -4.3729285e-04,  2.7476146e-05],
        ...,
        [ 7.5996635e-05,  8.8360357e-05, -2.3720469e-05, ...,
          7.3127999e-06, -3.0145174e-04, -8.6266809e-06],
        [ 7.8365869e-05,  3.8380971e-05, -1.1082434e-06, ...,
         -3.2321209e-04, -3.3568274e-03, -1.2811105e-04],
        [-5.5836072e-06, -5.5833748e-06, -7.5000353e-06, ...,
          4.4880211e-04,  6.2548197e-03,  6.2306615e-05]], dtype=float32)]

In [0]:
tracks.head()

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity,artists_enc,artist_enc
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",-0.203933,0.834130,238373.0,-0.355265,0.016552,-1.126213,-0.666719,0.293658,0.832017,2.359999,2.433574,4.0,-0.173393,44,31054,31054.0
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),-0.202371,1.375677,214800.0,-0.149769,0.016552,0.798000,0.562219,0.356709,0.832017,2.743943,1.191883,4.0,-0.173386,10,31054,31054.0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,-0.202321,0.098047,138913.0,0.006709,0.016552,1.072888,-0.659518,0.562718,-1.203850,-0.548370,-0.025620,4.0,-0.173386,63,23118,23118.0
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,-0.201951,1.133821,125381.0,-0.129031,0.510755,-0.026663,-0.557507,-0.374477,-1.203850,-0.350799,0.196551,4.0,-0.173378,9,5309,5309.0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,-0.204420,1.044440,124016.0,0.071751,0.492331,0.523112,-0.954752,-0.097442,0.832017,-0.382794,0.114836,4.0,-0.173370,8,5309,5309.0


In [0]:
# now we can finally predict!!
# lets choose a random track from the training df so I don't keep using Tame Impala :)
random_number = np.random.randint(1, tracks.shape[0]+1)
lucky_track = tracks.loc[random_number]
print('Track Name:', lucky_track[2])
print('Artist Name:', lucky_track[0])
print('Track ID:', lucky_track[1])
# cool looks like we got a classical music track

Track Name: Franck / Arr. Kenner: Mass in A Major, Op. 12, FWV 61: V. Panis Angelicus
Artist Name: César Franck
Track ID: 3Xb1RZj2KP4rADqKf2CWGw


In [0]:
# prediction dun dun dunnnnn
boom = model.predict(num_features.loc[random_number].values.reshape(1,num_features.shape[1]))

In [0]:
top10 = tracks.loc[boom.argsort()[0][-10:]]
top10[['track_name', 'artist_name', 'track_id']]
# cool, looks like a classical track
# the first 3 recs, and recs 7-9 are music but NOT classical genre
# rec 4 is a speaker/headphone test, rec 6 is sleep aid track
# we do have 3 tracks by Bach! 

,track_name,artist_name,track_id
52809,20 Miles,Shy Martian,2ruTVwK7cwFNURaY2DSTVG
24460,Tell Me,Kayla Jay,5S7zmrPk49tNGiRUBoydiu
33715,War and Church,DJ Straight G,5A91GwNl0ONDLMLsrOgCH5
83476,Competion Bass (Bass Test Extreme),Flunky Junky,3xFiCijZzjvhcOZJqU4nVQ
3134,"無伴奏ヴァイオリン・ソナタ 第 1番 ト短調, BWV 1001: 4. Presto",Johann Sebastian Bach,5y2ffpJMeB2G9YgKAx65wd
3125,"Orchestral Suite No. 2 in B Minor, BWV 1067: V...",Johann Sebastian Bach,4brUvpj2jUSC4iwsk1TUf4
83449,Soothing Music for Sleep and Delta Waves,Sleeping Music Experience,0Pf7JLTpwGmAL586tCtw5t
92673,Blueberry (Mix Cut) - Daniel Wanrooy Remix,Probspot,6szH9pUUWuYFNNMckB1SCS
100799,"FIND YOUR WINGS (feat. Roy Ayers, Sydney Benne...","Tyler, The Creator",0mvnmLsfi9n5RjPIWzT6tY
99787,"Befiehl du deine Wege, BWV 271",Johann Sebastian Bach,1NXyNlJnwRyzqH2SegKQYF


In [0]:
# lets check out Tame Impala 'Borderline'
boom2 = model.predict(num_features.loc[103658].values.reshape(1,num_features.shape[1]))

In [0]:
top10 = tracks.loc[boom2.argsort()[0][-10:]]
top10[['track_name', 'artist_name', 'track_id']]
# this is pretty bad, almost none of these tracks are similar genre
# 1st rec is a relaxation/sleeping aid track
# 4 recs are rap
# 1 rec is a mostly speech album intro

,track_name,artist_name,track_id
47326,Crashing Waves,Markup,3raPV6mhoF57MX14CbiyDA
47324,Acid Chunk - Original Mix,Weiss (UK),4Xae0PVpAG7oKK6JMphrXp
47321,Dont Let Me Fall,Zeper,4mIkBg0Xm0fZrEuUrwqWuB
47320,Duress,AOBeats,45dD2M6BjqU3m0aVeXZ9XI
47318,Loathe,Nate Rose,2R7RwGR0jUEieQigHVHl6d
47317,Rooftop,Young Syrup,1ceT2qUNievutaLNCn4Zf9
47316,Goodbye - The John F. Kennedy Center for The P...,Chicago,6bRm4NOttsiQOYnfCvmLSI
47314,"You Will Be Found (From ""Dear Evan Hansen"")",Benj Pasek,3iFjfGxlPGFjavrtVaDGKx
47334,Glitter,Daniel Avery,6qDe01ZfTYi9YpQwNA4sT0
59961,Two of a Kind,Ghost and Pals,5x2QvZQxO7QVnXMhpR9vxK


In [0]:
tracks.loc[111644]

artist_name                 Britney Spears
track_id            4fbaKWFRghusXd4bSBvvfN
track_name                           Toxic
acousticness                     -0.202329
danceability                       0.99712
duration_ms                         198800
energy                            0.115112
instrumentalness                 0.0300993
key                             -0.0266629
liveness                          0.298189
loudness                          0.860068
mode                              -1.20385
speechiness                    0.000348906
tempo                             0.750443
time_signature                           4
valence                           -0.17337
popularity                              67
artists_enc                           4090
artist_enc                            4090
Name: 111644, dtype: object

In [0]:
# 'Toxic' Britney Seaprs
boom3 = model.predict(num_features.loc[111644].values.reshape(1,num_features.shape[1]))

In [0]:
top10 = tracks.loc[boom3.argsort()[0][-10:]]
top10[['track_name', 'artist_name', 'track_id']]
# this model seems much more poor than our KNN unfortunately

,track_name,artist_name,track_id
329,Volvamos Hablar,Bryant Myers,1sLUxTb7yh97dI3qiuijpj
41885,"Dearly Beloved (From ""Kingdom Hearts"")",Myuu,5GEQiL5XmC6FpJVNOtaYwJ
22329,Never Enough (From the Greatest Showman),Malinda Kathleen Reese,4w8ZCkXrdxie4NzFygD64a
66500,sneakerhead,underscores,26hcTYdnaPjVC1rIPpQYLm
79626,On My Way,Shiah Maisel,4qDj2LHFQpket2tAxhAHIh
54851,The Konnokal - N3bula & Twisted Melodiez Remix,Dimatik,5a7jAlB8ShXHq5S9Kx4YNJ
60853,Wanna Dance,Steve Davit,7KmI92MtxCqooGCcK6TB4u
58816,Snow,Michel Tuati,2yzKmT8plSJk4aT08afqX2
113489,World Wide,700prince,7bsDkKlVNeqhFI2mAVEoIq
115611,Panacea.,Misc.Inc,3julkyNbFWkLu9geA2CB8K


In [0]:
# make functions for all of this for implementation
def predict_tracks(model, vector):
  return model.predict(vector).argsort()

In [0]:
def create_model(weights):
  model = Sequential()
  model.add(Dense(input_shape=(12,),
          activation='relu',
          units=weights.shape[0],
          use_bias=False))
  model.set_weights([weights.T])
  return model

In [0]:
def model_result(vector, features, best_match=True, amount=11):
    """
    fxn which takes input vector (track features) and all track features
    of the dataset and returns a list of 11 indices
    most similar if best_match=True, most dissimilar is best_match=False
    """
    tr_id = vector['id'].values[0]
    ids = features['id']
    input_vec = vector.drop('id', axis=1)
    feats = features.drop('id', axis=1)
    norm_vector = normie(input_vec.values.reshape(1,12))
    norm_features = normie(feats)
    model = create_model(norm_features)
    prediction = np.array(predict(model, norm_vector).argsort())
    prediction = prediction.reshape(prediction.shape[1])
    if best_match:
        if tr_id in ids[prediction[-amount:]]:
            return features.loc[prediction[-amount-1:-1]]
        return features.loc[prediction[-amount:]]
    return features.loc[prediction[:amount]]

In [0]:
test_result = model_result(track_audio_features, )

In [0]:
tracks[numeric_features]

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,-0.203933,0.834130,-0.355265,0.016552,-1.126213,-0.666719,0.293658,0.832017,2.359999,2.433574,-0.173393
1,-0.202371,1.375677,-0.149769,0.016552,0.798000,0.562219,0.356709,0.832017,2.743943,1.191883,-0.173386
2,-0.202321,0.098047,0.006709,0.016552,1.072888,-0.659518,0.562718,-1.203850,-0.548370,-0.025620,-0.173386
3,-0.201951,1.133821,-0.129031,0.510755,-0.026663,-0.557507,-0.374477,-1.203850,-0.350799,0.196551,-0.173378
4,-0.204420,1.044440,0.071751,0.492331,0.523112,-0.954752,-0.097442,0.832017,-0.382794,0.114836,-0.173370
...,...,...,...,...,...,...,...,...,...,...,...
120099,5.933602,0.723718,0.267819,-3.447207,-1.401101,-0.951151,1.453640,-1.082309,0.936210,-3.180850,4.448751
120100,5.008694,-0.259479,5.923650,-5.140612,-1.401101,-0.639716,1.565974,-1.203850,0.120331,-3.199869,4.750980
120101,5.260942,0.739491,0.267819,-3.825987,-1.126213,-0.835338,1.452060,0.512386,0.072338,-3.197215,4.439552
120102,4.672364,-0.417211,5.923650,-6.412427,-1.126213,-0.878543,1.493141,0.439095,-0.287608,-3.202675,5.967585


###We decided not to implement this NN

In [0]:


# train validate
target = 'track_id'
features = tracks[numeric_features]

X_train = features
y_train = tracks[target]

In [0]:
# random
seed = 42 
numpy.random.seed(seed)


# supercalafragalisticexpialadocious parameters
batch_size = 512
epochs = 10
optimizer = 'Adam'

# instantiate
model = Sequential()
mode.add(Dense(12, imput_shape=12, activation = 'relu'))
model.add(Dense(512, activation = 'sigmoid'))
model.add(Dense(1024, activation = 'sigmoid'))
model.add(Dense(512, activation = 'sigmoid'))
